In [18]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [20]:
from langchain_teddynote import logging

# API 키를 명시적으로 전달
logging.langsmith(project_name="test")
# logging.langsmith("test", set_enable=False)

LangSmith 추적을 시작합니다.
[프로젝트명]
test


In [21]:
from langchain_teddynote.messages import stream_response
from langchain_core.prompts import PromptTemplate

In [22]:
# template 정의
template = "{country}의 수도는 어디인가요?"

# from_template 메소드를 이용하여 PromptTemplate 객체 생성
prompt_template = PromptTemplate.from_template(template)
prompt_template

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?')

In [23]:
# prompt 생성
prompt = prompt_template.format(country="대한민국")
prompt

'대한민국의 수도는 어디인가요?'

In [25]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-4o-mini",
    max_tokens=2048,
    temperature=0.1,
)

In [ ]:
prompt = PromptTemplate.from_template("{topic} 에 대해 쉽게 설명해주세요.")
model = ChatOpenAI()
chain = prompt | model

In [29]:
# input 딕셔너리에 주제를 '인공지능 모델의 학습 원리'으로 설정합니다.
input = {"topic": "인공지능 모델의 학습 원리"}

In [30]:
# prompt 객체와 model 객체를 파이프(|) 연산자로 연결하고 invoke 메서드를 사용하여 input을 전달합니다.
# 이를 통해 AI 모델이 생성한 메시지를 반환합니다.
chain.invoke(input)

AIMessage(content='인공지능 모델의 학습 원리는 데이터를 기반으로 모델이 일정한 규칙이나 패턴을 찾아내는 과정입니다. \n\n모델은 입력 데이터와 출력 데이터 사이의 관계를 학습하여, 주어진 입력에 대해 올바른 출력을 예측할 수 있도록 학습됩니다. 이때 모델은 초기에는 임의의 가중치와 편향을 가지고 시작하며, 학습을 거치면서 이 가중치와 편향을 조정해가며 더욱 정확한 예측을 하도록 학습합니다.\n\n주어진 데이터와 모델의 예측값과의 차이를 최소화하는 방향으로 학습을 진행하며, 이를 위해 손실 함수를 정의하여 모델이 얼마나 잘 예측했는지 측정합니다. 손실 함수를 최소화하는 방향으로 모델의 가중치와 편향을 업데이트하면서 반복적으로 학습을 진행합니다.\n\n이러한 학습 과정을 통해 모델은 주어진 데이터에 대해 정확한 예측을 하도록 학습되고, 새로운 데이터에 대해서도 유사한 패턴을 찾아내어 예측을 수행할 수 있게 됩니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 399, 'prompt_tokens': 33, 'total_tokens': 432, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-8016d70f-08c6-4024-8108-dbb7303cf89d-0', usage_metadata={'input_to

In [31]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

인공지능 모델의 학습 원리는 데이터를 입력으로 받아 이를 처리하여 원하는 결과를 출력하는 과정입니다. 이 모델은 초기에는 데이터를 입력 받고, 이를 분석하고 패턴을 학습하여 문제를 해결하는 방법을 습득합니다. 학습 과정에서 모델은 오차를 줄이기 위해 다양한 가중치를 조정하고 최적의 결과를 찾아내는 작업을 수행합니다. 모델은 학습 데이터를 계속 학습하면서 점차 더 정확한 결과를 출력할 수 있도록 성능을 향상시키게 됩니다. 이러한 방식으로 인공지능 모델은 데이터를 학습하여 필요한 작업을 수행하고 문제를 해결하는 것이 가능해집니다.

In [38]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel

# 예: PydanticOutputParser를 사용하여 모델 출력을 Pydantic 객체로 변환
class WeatherResponse(BaseModel):
    temperature: float
    description: str

parser = PydanticOutputParser(pydantic_object=WeatherResponse)
model_output = '{"temperature": 25.3, "description": "Clear sky"}'
parsed_output = parser.parse(model_output)

print(parsed_output.temperature)  # 25.3
print(parsed_output.description)  # "Clear sky"

25.3
Clear sky


In [39]:
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

In [40]:
# 프롬프트, 모델, 출력 파서를 연결하여 처리 체인을 구성합니다.
chain = prompt | model | output_parser

In [41]:
# chain 객체의 invoke 메서드를 사용하여 input을 전달합니다.
input = {"topic": "인공지능 모델의 학습 원리"}
chain.invoke(input)

'인공지능 모델의 학습 원리는 데이터를 통해 패턴을 학습하고 예측하는 과정을 말합니다. \n\n먼저, 모델은 훈련 데이터를 입력으로 받아 해당 데이터에 내재된 패턴이나 관계를 학습합니다. 이를 위해 모델은 입력 데이터와 이에 대응하는 출력 값을 비교하면서 오차를 줄여가는 과정을 거치게 됩니다. \n\n이러한 과정을 통해 모델은 입력 데이터의 특징을 파악하고 이를 기반으로 새로운 데이터에 대해 예측을 수행할 수 있게 됩니다. 이러한 학습 원리를 통해 인공지능 모델은 데이터를 이해하고 효율적으로 활용할 수 있게 되는 것입니다.'

In [42]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

인공지능 모델의 학습 원리는 데이터를 이용하여 패턴을 학습하는 과정입니다. 먼저 모델은 입력 데이터를 받아들이고, 이 데이터를 분석하여 패턴을 찾습니다. 그리고 이러한 패턴을 기반으로 적절한 출력을 생성하도록 학습을 진행합니다. 모델은 학습 데이터를 반복적으로 분석하고 패턴을 강화하여 정확한 결과를 얻을 수 있도록 학습을 진행합니다. 이러한 과정을 통해 모델은 더 나은 예측이나 결정을 내릴 수 있게 되는 것이죠.요약하면, 인공지능 모델의 학습 원리는 데이터를 이용하여 패턴을 학습하는 과정입니다.

In [43]:
template = """
당신은 영어를 가르치는 10년차 영어 선생님입니다. 상황에 [FORMAT]에 영어 회화를 작성해 주세요.

상황:
{question}

FORMAT:
- 영어 회화:
- 한글 해석:
"""

# 프롬프트 템플릿을 이용하여 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(template)

# ChatOpenAI 챗모델을 초기화합니다.
model = ChatOpenAI(model_name="gpt-4o-mini")

# 문자열 출력 파서를 초기화합니다.
output_parser = StrOutputParser()

In [44]:
# 체인을 구성합니다.
chain = prompt | model | output_parser

# 완성된 Chain을 실행하여 답변을 얻습니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "저는 식당에 가서 음식을 주문하고 싶어요"})
# 스트리밍 출력
stream_response(answer)

- 영어 회화:
  - Waiter: Good evening! Welcome to our restaurant. How many people are in your party?
  - You: Hi! It’s just me tonight.
  - Waiter: Great! Please follow me to your table. Here’s the menu. Can I get you something to drink?
  - You: Yes, I’d like a glass of water, please.
  - Waiter: Of course! Are you ready to order your meal?
  - You: Not yet, can I have a few more minutes?
  - Waiter: Absolutely, take your time. Let me know if you have any questions about the menu.
  - You: Thank you!
  - (After a few minutes)
  - You: I’m ready to order. I’d like the grilled chicken with a side salad, please.
  - Waiter: Excellent choice! Would you like any dressing on your salad?
  - You: Yes, I’ll have the balsamic vinaigrette.
  - Waiter: Perfect! I’ll get that for you right away.

- 한글 해석:
  - 웨이터: 좋은 저녁입니다! 저희 식당에 오신 것을 환영합니다. 몇 분이세요?
  - 당신: 안녕하세요! 오늘은 저 혼자입니다.
  - 웨이터: 좋습니다! 이쪽으로 따르세요. 여기 메뉴입니다. 음료수는 무엇을 드릴까요?
  - 당신: 네, 물 한 잔 부탁드려요.
  - 웨이터: 물론입니다! 식사를 주문할 준비가 되셨나요?
  - 당신: 아직 아닙니